In [1]:
from tensorflow import keras
from keras_preprocessing.text import Tokenizer  #type: ignore
from keras.layers import Embedding, LSTM, Dense
from keras.models import Sequential
from keras.utils import to_categorical
from keras.optimizers import Adam
import pickle
import numpy as np
import os

In [2]:
import keras
print(keras.__version__)


3.4.1


In [3]:
file  = open('data.txt', 'r', encoding = "utf8")

In [4]:
lines=[]
for i in file:
    lines.append(i)

In [5]:
data=""
for i in lines:
    data = ' '.join(lines)

In [6]:
data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('“','').replace('”','')

In [7]:
data = data.split()
data = ' '.join(data)

In [8]:
data[:1000]

'The Project Gutenberg eBook of Los Raros This ebook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions whatsoever. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this ebook or online at www.gutenberg.org. If you are not located in the United States, you will have to check the laws of the country where you are located before using this eBook. Title: Los Raros Author: Rubén Darío Illustrator: Enrique Ochoa Release date: November 1, 2015 [eBook #50365] Language: Spanish Credits: Produced by Josep Cols Canals, Chuck Greif and the Online Distributed Proofreading Team at http://www.pgdp.net (This file was produced from images generously made available by The Internet Archive/Canadian Libraries) *** START OF THE PROJECT GUTENBERG EBOOK LOS RAROS *** Produced by Josep Cols Canals, Chuck Greif and the Online Distributed Proofreading Team at http://www.pgdp

In [9]:
len(data)

400406

In [10]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

In [11]:
pickle.dump(tokenizer, open('token.pkl', 'wb'))

In [12]:
sequence_data = tokenizer.texts_to_sequences([data])[0]

In [13]:
sequence_data[:100]

[29,
 48,
 125,
 503,
 33,
 8,
 419,
 109,
 503,
 237,
 219,
 29,
 457,
 33,
 1356,
 3251,
 69,
 29,
 420,
 325,
 64,
 1357,
 366,
 3252,
 33,
 29,
 3253,
 421,
 16,
 2258,
 64,
 107,
 3254,
 16,
 3255,
 3256,
 60,
 422,
 562,
 423,
 1699,
 423,
 3257,
 58,
 3258,
 457,
 423,
 1145,
 29,
 274,
 33,
 29,
 48,
 125,
 342,
 2259,
 107,
 109,
 503,
 58,
 1146,
 421,
 620,
 125,
 756,
 275,
 60,
 256,
 201,
 984,
 69,
 29,
 420,
 325,
 60,
 862,
 985,
 50,
 1700,
 29,
 678,
 33,
 29,
 1701,
 1358,
 60,
 256,
 984,
 1702,
 1147,
 109,
 503,
 5410,
 8,
 419,
 5411,
 1148,
 1149,
 5412,
 1703]

In [14]:
len(sequence_data)

68022

In [15]:
vocab_size = len(tokenizer.word_index)+1

In [16]:
vocab_size

14297

In [17]:
sequence=[]
for i in range(10, len(sequence_data)):
    words = sequence_data[i-10:i+1]
    sequence.append(words)

In [18]:
len(sequence)

68012

In [19]:
sequence = np.array(sequence)

In [20]:
sequence

array([[   29,    48,   125, ...,   503,   237,   219],
       [   48,   125,   503, ...,   237,   219,    29],
       [  125,   503,    33, ...,   219,    29,   457],
       ...,
       [   64,  2256,    50, ...,    50, 14296,   983],
       [ 2256,    50, 14294, ..., 14296,   983,   894],
       [   50, 14294,    50, ...,   983,   894,  1140]])

In [21]:
X=[]
y=[]

for i in sequence:
    X.append(i[0:10])
    y.append(i[10])

In [22]:
X=np.array(X)
y=np.array(y)

In [23]:
y = to_categorical(y, num_classes=vocab_size)

In [24]:
y[:10]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

Building the LSTM Model

In [25]:
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length = 10))
model.add(LSTM(1000, return_sequences = True)) #return_sequences=True asegura que la capa devolverá una secuencia de salidas para cada paso de la entrada, necesario para la siguiente LSTM.
model.add(LSTM(1000))
model.add(Dense(1000, activation='relu'))
model.add(Dense(vocab_size, activation="softmax"))

/Users/gustavogutierrez/miniconda3/envs/Ambiente-Conda/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [26]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [27]:
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

# Compilar el modelo
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001))

early_stopping = EarlyStopping(monitor='val_loss', patience=5,restore_best_weights=True)

# Entrenar el modelo
model.fit(X, y, epochs=10, batch_size=64, callbacks=[early_stopping])  # Ajusta los epochs y batch_size según tus necesidades


Epoch 1/10
1063/1063 ━━━━━━━━━━━━━━━━━━━━ 395s 367ms/step - loss: 7.5918
Epoch 2/10
1063/1063 ━━━━━━━━━━━━━━━━━━━━ 416s 391ms/step - loss: 6.9351
Epoch 3/10
1063/1063 ━━━━━━━━━━━━━━━━━━━━ 494s 465ms/step - loss: 6.3954
Epoch 4/10
1063/1063 ━━━━━━━━━━━━━━━━━━━━ 487s 458ms/step - loss: 6.1062
Epoch 5/10
1063/1063 ━━━━━━━━━━━━━━━━━━━━ 487s 458ms/step - loss: 5.8045
Epoch 6/10
1063/1063 ━━━━━━━━━━━━━━━━━━━━ 483s 454ms/step - loss: 5.5049
Epoch 7/10
1063/1063 ━━━━━━━━━━━━━━━━━━━━ 492s 463ms/step - loss: 5.1918
Epoch 8/10
1063/1063 ━━━━━━━━━━━━━━━━━━━━ 491s 462ms/step - loss: 4.9142
Epoch 9/10
1063/1063 ━━━━━━━━━━━━━━━━━━━━ 483s 454ms/step - loss: 4.5878
Epoch 10/10
1063/1063 ━━━━━━━━━━━━━━━━━━━━ 495s 466ms/step - loss: 4.2716


In [37]:
model.save("modelo_texto_Gustavo.keras")

In [38]:
def generate_text(seed_text, next_words, model, tokenizer, max_sequence_len=3):
    for _ in range(next_words):
        # Convertir el seed_text en una secuencia de enteros
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        
        # Asegurarse de que el token_list tenga exactamente max_sequence_len palabras
        token_list = token_list[-max_sequence_len:]
        
        # Rellenar o recortar la secuencia para que tenga max_sequence_len palabras
        token_list = np.pad(token_list, (max(0, max_sequence_len - len(token_list)), 0), mode='constant')

        # Predecir la próxima palabra
        predicted_probs = model.predict(token_list.reshape(1, max_sequence_len), verbose=0)
        predicted = np.argmax(predicted_probs, axis=-1)[0]
        
        # Obtener la palabra correspondiente al índice predicho
        output_word = tokenizer.index_word[predicted]
        
        # Agregar la palabra al texto semilla
        seed_text += " " + output_word

    return seed_text


In [42]:
seed_text = "Hay en estas páginas mucho entusiasmo, admiración sincera"
generated_text = generate_text(seed_text, next_words=20, model=model, tokenizer=tokenizer)

print()
print(generated_text)


Hay en estas páginas mucho entusiasmo, admiración sincera de la vida de los que se dejaron de los ojos de los cuales los prenden o el de los
